In [31]:
import numpy as np  
import pytest 

### A

In [1]:
def cipher(text, shift, encrypt=True):
    alphabet = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'
    new_text = ''
    for c in text:
        index = alphabet.find(c)
        if index == -1:
            new_text += c
        else:
            new_index = index + shift if encrypt == True else index - shift
            new_index %= len(alphabet)
            new_text += alphabet[new_index:new_index+1]
    return new_text

In [6]:
def test_word():
    example = 'word'
    shift = 4
    expected = 'Asvh'
    actual = cipher(example, shift)
    assert actual == expected


test_word()

## B

In [9]:
def test_neg():
    test_text = "The New York Mets were once good at baseball and then they sucked"
    example_shift = -2
    expected = "Rfc Lcu Wmpi Kcrq ucpc mlac emmb Yr ZYqcZYjj Ylb rfcl rfcw qsaicb"
    actual = cipher(test_text, example_shift)
    assert actual == expected

test_neg()

## C

In [13]:
def test_special():
    test_text = "UhOh!?!%"
    test_shift = 5
    expected = "ZmTm!?!%"
    actual = cipher(test_text, test_shift)
    assert expected == actual

test_special()


## D

In [2]:
def cipher(text, shift, encrypt=True):
    assert type(shift) != str, "Cannot pass string to shift."
    alphabet = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'
    new_text = ''
    for c in text:
        index = alphabet.find(c)
        if index == -1:
            new_text += c
        else:
            new_index = index + shift if encrypt == True else index - shift
            new_index %= len(alphabet)
            new_text += alphabet[new_index:new_index+1]
    return new_text




In [23]:
cipher("two", "two")

AssertionError: Cannot pass string to shift. 

In [32]:
def test_assert():
    with pytest.raises(Exception):
        test_word = "two"
        test_shift = "two"
        cipher(test_word,test_shift)


In [33]:
test_assert()

## E

In [9]:
!pytest -vv test_cipher.py

============================= test session starts ==============================
platform darwin -- Python 3.9.1, pytest-7.1.3, pluggy-1.0.0 -- /Users/jacksonrudoff/.pyenv/versions/3.9.1/bin/python3
cachedir: .pytest_cache
rootdir: /Users/jacksonrudoff/Documents/QMSS/Fall_2022/mds_course/Rudoff_Jackson/hw06
plugins: Faker-15.0.0
collected 4 items                                                              

test_cipher.py::test_word PASSED                                         [ 25%]
test_cipher.py::test_neg PASSED                                          [ 50%]
test_cipher.py::test_special PASSED                                      [ 75%]
test_cipher.py::test_assert PASSED                                       [100%]

============================== 4 passed in 0.03s ===============================
